In [2]:
import time
from Arm_Lib import Arm_Device
import roboticstoolbox as rtb
from roboticstoolbox import *
from spatialmath import *
import math
from math import pi
import numpy as np
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import os
import threading
from spatialmath import SE3
from spatialmath.base import e2h,h2e
from machinevisiontoolbox import CentralCamera #针孔相机模型

%matplotlib widget
%matplotlib widget
Arm  = Arm_Device()

Part1：Robot Configuration

In [3]:
"""DH标定"""
#相机DH
DFbot1 = DHRobot(
    [
        RevoluteMDH(d=0.04145,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(alpha=np.pi/2,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteMDH(alpha=-np.pi/2,qlim=np.array([0,np.pi])),
        RevoluteMDH(a=0.0481,d=0.0657,qlim=np.array([-np.pi,np.pi]))
    ],
    name = "DFbot",
)

#夹爪DH
DFbot2 = DHRobot(
    [
        RevoluteDH(d=0.04145,alpha=np.pi/2,qlim=np.array([-np.pi/2,np.pi/2])),
        RevoluteDH(a=-0.08285,qlim=np.array([-np.pi,0])),
        RevoluteDH(a=-0.08285,qlim=np.array([-np.pi/2,np.pi/2])),
        RevoluteDH(alpha=-np.pi/2,qlim=np.array([0,np.pi])),
        RevoluteDH(d=0.18,qlim=np.array([-np.pi/2,3*np.pi/2]))
    ],
    name = "DFbot",
)

In [5]:
#夹爪开合角 (degree)
claw_open = 150
claw_close = 45

In [6]:
#常见关节空间
vertical = [0,-90,0,90,0,claw_open]
origin = [0,0,0,0,0,claw_close]
cam_calibration=[0,-180,90,180,0,claw_close]
photo1 = [0,-120,90,180,0,claw_open]
photo2 = [0,-60,60,180,0,claw_open]

In [7]:
"""相机内参
"""
cam = CentralCamera(f=[934.44192173e-6,939.51291465e-6],rho = 1e-6, imagesize=[640,480], pp=[333.2247427,171.10048061])

mtx = cam.K

dist = np.array([[-5.35651039e-01,  1.53009433e+00,  1.47216506e-04,
        -2.40415479e-03, -3.83108955e+00]])

Part2: Basic Functions

In [8]:
"""角度弧度转换:degreeToRadian和RadianToDegree
"""
def degreeToRadian(degree_list):
    return [math.radians(degree) for degree in degree_list]
def radianToDegree(radian_list):
    return [math.degrees(radian) for radian in radian_list]

In [9]:
"""获取当前关节角：readJointAngles
功能：获取当前（5或6）关节角度，选择角度或者弧度
返回：list
"""
def readJointAngles(unit="degree",joint=5):
    
    current_position = []
    for i in range(joint):
            angle=Arm.Arm_serial_servo_read(i+1)
            if angle is None:
                raise ValueError(f"Failed to read current position for joint {i+1}")
            current_position.append(angle)
        
    #角度处理
    current_position[0] = +current_position[0] - 90
    current_position[1] = -current_position[1]
    current_position[2] = -current_position[2] + 90
    current_position[3] = -current_position[3] + 180
    current_position[4] = +current_position[4] - 90
    if joint==6: current_position[5] = 180-current_position[5]

    #转换弧度
    if unit=="radian":
        return degreeToRadian(current_position)
        
    return current_position

In [10]:
"""Direct_control: 直接对机械臂进行带速度限制的6关节控制
输入：形状为（6，）的六关节角度，in degree
功能：关节限位检测，运动速度限制，机械臂按照关节空间运动
返回：抵达的6关节空间，运动时间（ms）
"""
def Direct_control(joint_angles):#joint_angle为一个（6，）的numpy array

    # 确保输入是numpy数组
    joint_angles = np.asarray(joint_angles)
    
    # 检查输入形状是否正确
    if joint_angles.shape != (6,):
        raise ValueError("The shape of input angles must be 6!")
    
    # 计算新的关节角
    j1 = +joint_angles[0] + 90
    j2 = -joint_angles[1]
    j3 = -joint_angles[2] + 90
    j4 = -joint_angles[3] + 180
    j5 = +joint_angles[4] + 90
    j6 = -joint_angles[5] + 180 #input的角越小，夹爪越收紧！

    #关节限位检测
    if (j1<0)or(j1>180):
        raise ValueError("Joint angle 1 exceeds the range!")
    elif (j2<0)or(j2>180):
        raise ValueError("Joint angle 2 exceeds the range!")
    elif (j3<0)or(j3>180):
        raise ValueError("Joint angle 3 exceeds the range!")
    elif (j4<0)or(j4>180):
        raise ValueError("Joint angle 4 exceeds the range!")
    elif(j5<0)or(j5>270):
        raise ValueError("Joint angle 5 exceeds the range!")
    elif(j6<0)or(j6>180):
        raise ValueError("Joint angle 6 exceeds the range!")

    #限制运动速度：
    target = np.array([j1,j2,j3,j4,j5,j6])
    current_position = []
    for i in range(6):
            angle=Arm.Arm_serial_servo_read(i+1)
            if angle is None:
                raise ValueError(f"Failed to read current position for joint {i+1}")
            current_position.append(angle)
    angle_difference = np.abs(np.array(current_position)-target)
    biased_angle_difference = angle_difference.copy()
    biased_angle_difference[1:4] *= 3 #给2，3，4关节时间更长，给1，5，6关节时间可以稍微短
    max_angle_movement = np.max(biased_angle_difference)
    time = max_angle_movement * 1000 / 60

    Arm.Arm_serial_servo_write6(j1,j2,j3,j4,j5,j6, time.astype(int))
    return [j1,j2,j3,j4,j5,j6],time

In [11]:
"""计算FK和仿真：calculateFK,poseSimu
输入：（5或6）关节角度，in degree，相机或夹爪
功能：计算和仿真FK
返回：Transform matrix或仿真
"""
def calculateFK(joint_angles,DH=2):

    joint_angles = degreeToRadian(joint_angles)
    motor_angles = joint_angles[:5]
    
    if DH == 1: return DFbot1.fkine(np.hstack([motor_angles,0]))
        
    else: 
        return DFbot2.fkine(motor_angles)

def poseSimu(joint_angles,DH=2):
    joint_angles = degreeToRadian(joint_angles)
    motor_angles = joint_angles[:5]
    
    if DH == 1: DFbot1.plot(np.hstack([motor_angles,0]))
    else: DFbot2.plot(motor_angles)
    return

In [12]:
"""计算IK:calculateIK
输入：夹爪的位姿矩阵，初始关节空间，ilimit，slimit，joint_limit
功能：通过ik-LM优化算法计算距离q0最近的关节空间
返回：（degree）五关节的关节空间
"""
def calculateIK(T1,q0=[0,0,0,0,0],slimit=1000,ilimit=5000,tol=0.002,joint_limit=True):
    T1 = np.array(T1)
    if T1.shape[0]!=4 or T1.shape[1]!=4:
        raise ValueError("The input array has wrong shape!")
    #sol = DFbot2.ikine_LM(T1,q0,slimit=slimit,ilimit=ilimit,tol=tol,joint_limits=joint_limit,mask=[50,10,20,0.1,0.1,0.1])
    sol = DFbot2.ikine_LM(T1,q0,slimit=slimit,ilimit=ilimit,tol=tol,joint_limits=joint_limit,mask=[30,30,10,0.1,0.1,0.1])

    if sol.reason=='Success':
        return radianToDegree(sol.q)
    else: raise ValueError("Cannot find a solution!") 

In [13]:
"""普通视频串流与拍照：start_stream,capture_photo,stop_stream
功能：start_stream唤起普通视频串流
     capture_photo抓拍，可以传入filename更改名称
     stop_stream停止并释放串流
"""
# Create output directory if it doesn't exist
os.makedirs('captured_photos', exist_ok=True)

def start_stream():
    """Start video streaming from webcam"""
    global camera, image_widget, preview_active
    
    # Initialize camera
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        raise Exception("Could not open camera")
    
    # Create image widget for display
    image_widget = widgets.Image(format='jpeg', width=600, height=500)
    display(image_widget)
    
    preview_active = True
    
    def update_frame():
        """Update the displayed frame"""
        while preview_active:
            ret, frame = camera.read()
            if not ret:
                print("Failed to grab frame")
                break
            # Convert frame to JPEG and display
            _, jpeg = cv2.imencode('.jpg', frame)
            image_widget.value = jpeg.tobytes()
            time.sleep(0.033)  # ~30 FPS
    
    # Start the streaming in a thread
    import threading
    threading.Thread(target=update_frame, daemon=True).start()
    
    print("Streaming started - use stop_stream() to stop")

def capture_photo(filename='photo'):
    """Capture and save a photo from the webcam"""
    global camera
    
    # Check if camera is available
    if 'camera' not in globals() or not camera.isOpened():
        print("Camera not available - start streaming first")
        return
    
    ret, frame = camera.read()
    if ret:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{filename}_{timestamp}.jpg"
        filepath = os.path.join('captured_photos', filename)
        
        cv2.imwrite(filepath, frame)
        print(f"Photo saved as: {filepath}")
        
        # Display the captured photo
        '''
        _, jpeg = cv2.imencode('.jpg', frame)
        display(widgets.Image(value=jpeg.tobytes(), width=300))
        '''
    else:
        print("Failed to capture photo")

def stop_stream():
    """Stop video streaming and release camera"""
    global preview_active, camera
    
    if 'preview_active' in globals():
        preview_active = False
    
    if 'camera' in globals() and camera.isOpened():
        camera.release()
        print("Streaming stopped and camera released")
    else:
        print("No active streaming to stop")

In [14]:
"""校正图像畸变 undistort_image
参数:
    image: 输入图像
    mtx: 相机内参矩阵
    dist: 畸变系数
返回:
    校正后的图像
"""
def undistort_image(image, mtx, dist):
    h, w = image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    dst = cv2.undistort(image, mtx, dist, None, newcameramtx)
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

In [15]:
"""detect_color_object:带有物块识别的视频串流
输入：color为物块颜色
功能：识别各个颜色物块边界，按“c”捕捉四个角及中间点，按esc退出,可以选择开启或关闭防畸变功能（开启会切割画面）
返回：
print(f"左上: {points[0]}")
print(f"右上: {points[1]}")
print(f"右下: {points[2]}")
print(f"左下: {points[3]}")
print(f"中心: {points[4]}")
"""

def detect_color_object(color='red'):
    """
    颜色物体检测函数
    参数:
        color: 要检测的颜色 ('red', 'blue' 或 'green')
    返回:
        当按下'c'时返回捕捉到的五个点坐标 (四个角点+中心点)
    """
    # 颜色阈值定义
    color_dist = {
        'red': [
            {'Lower': np.array([0, 100, 100]), 'Upper': np.array([10, 255, 255])},
            {'Lower': np.array([160, 100, 100]), 'Upper': np.array([180, 255, 255])}
        ],
        'blue': {'Lower': np.array([70, 70, 70]), 'Upper': np.array([124, 255, 255])},
        'green': {'Lower': np.array([35, 43, 35]), 'Upper': np.array([90, 255, 255])},
    }

    cap = cv2.VideoCapture(0)
    cv2.namedWindow("camera", cv2.WINDOW_AUTOSIZE)
    if color=="red": cv2.namedWindow("mask", cv2.WINDOW_AUTOSIZE)  # 新增mask窗口用于调试
    
    captured_points = None
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Cannot read camera!")
            break

        #畸变修正处理
        #frame = undistort_image(frame,mtx,dist)
        
        # 预处理
        gs_frame = cv2.GaussianBlur(frame, (5, 5), 0)
        hsv = cv2.cvtColor(gs_frame, cv2.COLOR_BGR2HSV)
        erode_hsv = cv2.erode(hsv, None, iterations=2)
        
        # 颜色阈值处理
        if color == 'red':
            lower1 = color_dist[color][0]['Lower']
            upper1 = color_dist[color][0]['Upper']
            lower2 = color_dist[color][1]['Lower']
            upper2 = color_dist[color][1]['Upper']
            
            mask1 = cv2.inRange(erode_hsv, lower1, upper1)
            mask2 = cv2.inRange(erode_hsv, lower2, upper2)
            inRange_hsv = cv2.bitwise_or(mask1, mask2)
            inRange_hsv = cv2.dilate(inRange_hsv, None, iterations=1)
        else:
            inRange_hsv = cv2.inRange(erode_hsv, 
                                     color_dist[color]['Lower'], 
                                     color_dist[color]['Upper'])
        
        if color=="red": cv2.imshow('mask', inRange_hsv)
        
        # 查找轮廓
        cnts = cv2.findContours(inRange_hsv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        
        if cnts:
            cnts = [c for c in cnts if cv2.contourArea(c) > 500]
            
            if cnts:
                c = max(cnts, key=cv2.contourArea)
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                
                # 绘制边界框和点
                cv2.drawContours(frame, [box], -1, (0, 255, 255), 2)
                
                # 绘制四个角点坐标
                points = []
                for i, point in enumerate(box):
                    x, y = point
                    points.append((x, y))
                    cv2.putText(frame, f"{x},{y}", (x, y), 
                               cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.75, (0, 255, 255), 1)
                
                # 计算中心点
                center_x = int((box[3][0] + box[1][0]) / 2)
                center_y = int((box[3][1] + box[1][1]) / 2)
                center = (center_x, center_y)
                points.append(center)
                cv2.putText(frame, f"{center_x},{center_y}", center,
                           cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.75, (0, 255, 255), 1)
                
                # 检查是否按下'c'键捕获坐标
                key = cv2.waitKey(1)
                if key == 99:  # 'c'键
                    captured_points = points
                    break
                elif key == 27:  # ESC键
                    break
        
        cv2.imshow('camera', frame)
        key = cv2.waitKey(1)
        if key == 27:  # ESC键
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return captured_points

In [16]:
"""关节插值和空间位姿插值控制：joint_interpolation,cartesian_interpolation
输入：
    关节插值：state1为目标6关节角（degree），state0读取当前关节角
"""
def joint_interpolation(state1,step=30):
    
    state0 = readJointAngles(joint=6,unit="radian")
    state1 = degreeToRadian(state1)

    qt = rtb.tools.trajectory.jtraj(state0,state1,step)

    for i in range(step):
        state = radianToDegree(qt.q[i])
        Direct_control(state)
        time.sleep(0.01)
    return qt.q

def cartesian_interpolation(T1,step=30):
    current_joint_angles = readJointAngles()
    T0 = calculateFK(current_joint_angles)
    if T1.shape[0]!=4 or T1.shape[1]!=4:
        raise ValueError("The input array has wrong shape!")
    print("Initial position:",current_joint_angles)
    qt = rtb.tools.trajectory.ctraj(T0,T1,step) #获得离散的T矩阵
    pose_list = []
    for i in range(len(qt)):
       # current_joint_angles = readJointAngles()
        pose = calculateIK(qt[i],q0=current_joint_angles,tol=0.005)
        pose_list.append(pose)
        Direct_control(np.hstack((pose,claw_close)))
        #time.sleep(0.01)
    return pose_list

In [17]:
"""欧拉角-旋转矩阵相互转换：matrix_to_euler, euler_to_matrix
输入：
    matrix_to_euler:其次变换矩阵T/旋转矩阵R
    euler_to_matrix:欧拉角（in degree）—— roll（绕x），pitch（绕y），yaw（绕z）
    旋转顺序按照roll-pitch-yaw，按照固定世界基底坐标系旋转！
输出：欧拉角，3x3旋转矩阵
"""
def matrix_to_euler(T):
    T = np.asarray(T)
    R = T[0:3,0:3]
    # 计算Pitch (θ)
    theta = math.atan2(-R[2, 0], math.sqrt(R[0, 0]**2 + R[1, 0]**2))
    
    # 处理万向节锁的情况
    if abs(theta - math.pi/2) < 1e-6:
        phi = 0
        psi = math.atan2(R[0, 1], R[1, 1])
    elif abs(theta + math.pi/2) < 1e-6:
        phi = 0
        psi = -math.atan2(R[0, 1], R[1, 1])
    else:
        # 计算Roll (φ)和Yaw (ψ)
        phi = math.atan2(R[2, 1], R[2, 2])
        psi = math.atan2(R[1, 0], R[0, 0])
    
    return radianToDegree([phi, theta, psi])

def euler_to_matrix(roll, pitch, yaw):
    roll = math.radians(roll)
    pitch = math.radians(pitch)
    yaw = math.radians(yaw)
    # 计算各旋转矩阵
    Rx = np.array([
        [1, 0, 0],
        [0, math.cos(roll), -math.sin(roll)],
        [0, math.sin(roll), math.cos(roll)]
    ])
    
    Ry = np.array([
        [math.cos(pitch), 0, math.sin(pitch)],
        [0, 1, 0],
        [-math.sin(pitch), 0, math.cos(pitch)]
    ])
    
    Rz = np.array([
        [math.cos(yaw), -math.sin(yaw), 0],
        [math.sin(yaw), math.cos(yaw), 0],
        [0, 0, 1]
    ])
    
    # 组合旋转矩阵 R = Rz * Ry * Rx
    R = np.dot(Rz, np.dot(Ry, Rx))
    return R

Part3: Task--Watch-Pick-Place

In [ ]:
Direct_control(photo2) #到达相机识别位姿
pose = calculateFK(photo2,DH=1)
depth = 0.115
K = mtx

In [ ]:
"""将3D世界坐标点投影到2D图像平面:project_3d_to_2d
参数:
    point_3d - 世界坐标系中的3D点(齐次坐标[x,y,z,1])
    K - 相机内参矩阵(3x3)
    pose - 相机位姿矩阵(4x4, 相机到世界的变换矩阵)
返回:
    point_2d - 图像平面上的2D坐标(像素坐标[u,v])
    depth - 深度值(相机坐标系下的z坐标)
"""
def project_3d_to_2d(point_3d, K, pose):
    # 确保所有矩阵都是numpy数组
    point_3d = np.hstack((np.array(point_3d),1))
    K = np.array(K)
    pose = np.array(pose)
    
    # 将3D点从世界坐标系转换到相机坐标系
    extrinsic = np.linalg.inv(pose)
    point_camera = extrinsic @ point_3d
    
    # 获取深度(相机坐标系下的z坐标)
    depth = point_camera[2]
    
    # 投影到图像平面
    point_2d_homogeneous = K @ point_camera[:3]
    point_2d = point_2d_homogeneous[:2] / point_2d_homogeneous[2]
    
    return point_2d, depth


In [ ]:
"""将2D图像点反投影到3D空间:backproject_2d_to_3d

参数:
    point_2d - 图像平面上的2D坐标(像素坐标[u,v])
    depth - 深度值(相机坐标系下的z坐标)
    K - 相机内参矩阵(3x3)
    pose - 相机位姿矩阵(4x4, 相机到世界的变换矩阵)

返回:
    point_3d - 世界坐标系中的3D点(齐次坐标[x,y,z,1])
"""
def backproject_2d_to_3d(point_2d, depth, K, pose):

    # 确保所有矩阵都是numpy数组
    K = np.array(K)
    pose = np.array(pose)
    
    # 将2D点转换为相机坐标系下的3D点
    point_2d_homogeneous = np.append(point_2d, 1)
    point_camera = depth * (np.linalg.inv(K) @ point_2d_homogeneous)
    
    # 转换为齐次坐标
    point_camera_homogeneous = np.append(point_camera, 1)
    
    # 转换到世界坐标系
    point_3d = pose @ point_camera_homogeneous
    
    return point_3d

In [ ]:
"""简化版抓取
条件：在可视范围内可以调整x，y，但是不能旋转方块
实现：通过识别方块中心点坐标，已知深度实现抓取
"""

#step1:获取中心点坐标
xaxis,yaxis = detect_color_object()[4] 
#step2:转换为空间3d坐标
new_point_3d = backproject_2d_to_3d([xaxis,yaxis],depth,K,pose) 
print("中心点：",new_point_3d[:3])

#step3:使用此位姿的旋转矩阵构造抓取的位姿
#pose2 = np.array(pose) #竖直抓取（成功率较低）
pose2 = np.array(calculateFK([0,-45,45,135,0])) #斜45度抓取(近处可能超出工作空间）
#pose2 = np.array(calculateFK([0,-45,45,150,0])) #斜60度抓取
pose2[:,-1] = new_point_3d.flatten()

#step4:计算抓取位姿的IK
sol = calculateIK(pose2,tol=0.01)
print("真实抓取 joint space:",sol)
print("真实抓取FK：\n",calculateFK(sol))

In [ ]:
#进行抓取(可人工对上面信息进行检查后运行）
Direct_control(np.hstack((sol,claw_open)))
time.sleep(2.5)
Direct_control(np.hstack((sol,claw_close)))

In [ ]:
"""完整版抓取
条件：在可视范围内可以调整x，y并旋转方块
实现：通过识别方块中心点坐标，已知深度实现抓取
"""

#step1:获取点坐标
points = np.array(detect_color_object())
xaxis,yaxis = points[4] 
#step2:转换为空间3d坐标
new_point_3d = backproject_2d_to_3d([xaxis,yaxis],depth,K,pose) 
print("中心点：",new_point_3d[:3])

#step3:使用此位姿的旋转矩阵构造抓取的位姿
pose2 = np.array(pose) #竖直抓取（成功率较低）
#pose2 = np.array(calculateFK([0,-45,45,135,0])) #斜45度抓取(近处可能超出工作空间）
#pose2 = np.array(calculateFK([0,-45,45,150,0])) #斜60度抓取
pose2[:,-1] = new_point_3d.flatten()

#step4:计算抓取位姿的IK
sol = calculateIK(pose2,tol=0.005)
print("真实抓取 joint space:",sol)
print("真实抓取FK：\n",calculateFK(sol))

#step5:计算物块旋转角度
theta1 = math.atan2(points[0,1]-points[1,1],points[0,0]-points[1,0])
theta2 = math.atan2(points[3,1]-points[2,1],points[3,0]-points[2,0])
theta = math.degrees((theta1+theta2)/2)
if theta > 90:
    theta = theta - 180
    if theta < -45:
        theta = theta + 90
print(theta1,theta2,"物块旋转角度：",theta)

In [ ]:
#进行抓取(可人工对上面信息进行检查后运行）
Direct_control(np.hstack((sol[0:4],theta,claw_open))) #末端6电机角度在这里修改
time.sleep(2.5)
#Direct_control(np.hstack((sol[0:4],35,claw_close)))

In [ ]:
#立方体投影仿真
from spatialmath import SE3
from machinevisiontoolbox import mkcube
X,Y,Z = mkcube(s=0.03,centre=(-0.15,0,-0.06),edge=True)
cam.plot_wireframe(X,Y,Z,pose=calculateFK(photo2,DH=1))

Part: Testing

In [ ]:
joint_interpolation(origin)

In [ ]:
cartesian_interpolation(calculateFK(vertical))

In [20]:
Direct_control(vertical)

([90, 90, 90, 90, 90, 30], 4500.0)

In [ ]:
readJointAngles(joint=6,unit="degree")

In [ ]:
a = calculateFK([-30,-45,35,140,60])
a

In [ ]:
calculateIK(a)

In [ ]:
poseSimu([-30,-45,35,140,60])

In [ ]:
b = calculateFK([0,0,0,-30,0],DH=2)
print(b)
b = np.array(b)

In [ ]:
poseSimu(origin,DH=1)

In [ ]:
calculateIK(a)

In [ ]:
start_stream()

In [ ]:
capture_photo()

In [ ]:
stop_stream()

In [ ]:
print(P)
print(project_3d_to_2d(P,K,pose))
backproject_2d_to_3d([138.73080609, 171.10048061],0.10750020470354073,K,pose)

In [ ]:
xaxis = 485
yaxis = 316
depth = 0.12
new_point_3d = backproject_2d_to_3d([xaxis,yaxis],depth,K,pose)
print(new_point_3d)

In [ ]:
pose2 = np.array(calculateFK([0,-45,45,150,0]))
pose2[:,-1] = new_point_3d.flatten()
pose2

In [ ]:
sol = calculateIK(pose2,tol=0.005)
print(sol)
calculateFK(sol)

In [ ]:
Direct_control([-5.311222614224444, -65.2227897980361, 82.87588050309691, 136.88339804839927, -4.988827395493744,claw_close])

In [ ]:
calculateFK([-0.7809924107131491,
 -57.64522352625729,
 59.39593307238916,
 177.9601331801728,
 -0.7791658064512503],DH=2)

In [ ]:
poseSimu([0,-45,45,150,0])
x = calculateFK([0,-45,45,150,0])
x

In [ ]:
matrix_to_euler(x)

In [ ]:
euler_to_matrix(0,-150,0)

In [19]:
detect_color_object()

[(96, 136), (203, 34), (304, 139), (197, 241), (200, 137)]

In [ ]:
#step1:获取中心点坐标
xaxis,yaxis = detect_color_object()[4] 
#step2:转换为空间3d坐标
new_point_3d = backproject_2d_to_3d([xaxis,yaxis],depth,K,pose) 

roll,pitch,yaw = 0,-180,34
R = euler_to_matrix(roll,pitch,yaw)
#step3:使用此位姿的旋转矩阵构造抓取的位姿
pose2 = np.array(pose) #竖直抓取（成功率较低）
#pose2 = np.array(calculateFK([0,-45,45,135,0])) #斜45度抓取(近处可能超出工作空间）
#pose2 = np.array(calculateFK([0,-45,45,150,0])) #斜60度抓取
pose2[:3,:3] = R
pose2[:,-1] = new_point_3d.flatten()
print(pose)
print(pose2)

#step4:计算抓取位姿的IK
sol = calculateIK(pose2,tol=0.015)
print("真实抓取 joint space:",sol)
print("真实抓取FK：\n",calculateFK(sol))